In [1]:
def evaluation_by_manual_registration(source,target,
                            
                             threshold = 0.03,
                             picked_id_source = None,
                             picked_id_target = None,
                                      
                             #statements
                             print_statements = False,
                             visualization_on = False
                             ):
    """
    transf,picked_id_source,picked_id_target = evaluation_by_manual_registration
    
    """
    
    

    if picked_id_source is None: 
        # pick points from two point clouds and builds correspondences
        picked_id_source = pick_points(source)
        
    if picked_id_target is None:
        picked_id_target = pick_points(target)
    
    print (picked_id_source)
    assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
    assert (len(picked_id_source) == len(picked_id_target))
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate rough transformation using correspondences
    if print_statements == True:
        print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))

    # point-to-point ICP for refinement
    if print_statements == True:
        print("Perform point-to-point ICP refinement")
    threshold = threshold  # 3cm distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    transf = reg_p2p.transformation 
    
    print (reg_p2p)
    if visualization_on == True: 
        draw_registration_result(source, target, transf)
        
    if print_statements == True:
        print("")
        print (reg_p2p)
        print(transf) 
    return transf,picked_id_source,picked_id_target,reg_p2p

In [ ]:
def full_evaluation_pipeline(
                             #input 
                             list_stitches,
                             labels_stitches = ["model_pc","stitched_pc"],
                             color_stitches = [[1, 0.706, 0],[0, 0.651, 0.929]],
                             
                             #plotting 
                             params = myparams,  # parameter for camera point view, json file via pressing P
                             configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = True,
                             rotate = False,
                             onewindow = True,
                             
                             #scaling
                             scale_factor = 1000,
                             voxels = [0.1,0.2],
                             
                             #evaluation 
                             threshold = 0.03,
                             picked_id_source = None,
                             picked_id_target = None,
    
                             #statements
                             print_statements = False,
                             visualization_on = False
                            ):
    """
    list_stitches = [pcd,st_pcd]
        pcd = original point cloud of the model
        st_pcd = our stitched point cloud that we want to evaluate against the model 
    
    
    """
    # check input size
    assert len(list_stitches) == len(labels_stitches) 
    assert len(labels_stitches) == len(color_stitches)
    assert len(color_stitches) == 2
    
    #
    pcd,st_pcd = list_stitches

    ## assign colors
    list_stitches[0].paint_uniform_color(color_stitches[0])
    list_stitches[1].paint_uniform_color(color_stitches[1])
    
    # mytitle
    mytitle = f"{labels_stitches[0]}_{get_color_name(color_stitches[0])}-{labels_stitches[1]}_{get_color_name(color_stitches[1])}"
    
    
    if visualization_on == True:
        # plot a list of geometries
        custom_draw_geometry(list_stitches[0]+list_stitches[1],
                             mytitle = mytitle+"before_resizing",
                             params = params,  # parameter for camera point view, json file via pressing P
                             configuration_file = configuration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = take_screen_shot,
                             rotate = rotate,
                             onewindow = onewindow)
    
    #define scaling function 
    trans_scale = np.asarray([[scale_factor, 0.0, 0.0, 0.0], 
                             [0.0, scale_factor, 0.0, 0.0],
                             [0.0, 0.0, scale_factor, 0.0], 
                             [0.0, 0.0, 0.0, 1.0]])
    
    # apply scaling function
    temp_pcd = copy.deepcopy(pcd)
    temp_pcd.transform(trans_scale)
    list_stitches[0] = temp_pcd
    
    if visualization_on == True:
        # plot a list of geometries
        custom_draw_geometry(list_stitches[0]+list_stitches[1],
                             mytitle = mytitle+"after_resizing",
                             params = params,  # parameter for camera point view, json file via pressing P
                             configuration_file = configuration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = take_screen_shot,
                             rotate = rotate,
                             onewindow = onewindow)
        
    #downsampling
    down_pcd = temp_pcd.voxel_down_sample(voxel_size=voxels[0])
    down_st_pcd = st_pcd.voxel_down_sample(voxel_size=voxels[1])
    downsampled_stitches = [down_pcd,down_st_pcd]
    
    if print_statements == True:
        print (len(down_pcd.points))
        print (len(down_st_pcd.points))
        
    #set source and target
    source = copy.deepcopy(downsampled_stitches[0])
    target = copy.deepcopy(downsampled_stitches[1])
    
    # evaluation by manual icp registration 
    transf,new_picked_id_source,new_picked_id_target, registration = evaluation_by_manual_registration(source,target,
                                                                             threshold = 0.03,
                                                                             picked_id_source = picked_id_source,
                                                                             picked_id_target = picked_id_target,
                                                                             visualization_on = visualization_on
                                                                            )
    
    return transf,new_picked_id_source,new_picked_id_target,registration,list_stitches